In [7]:
%pip install country_converter
%pip install pycountry_convert

Note: you may need to restart the kernel to use updated packages.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.1/10.1 MB 17.3 MB/s eta 0:00:0000:010:01
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.0/297.0 KB 4.7 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.7/98.7 KB 4.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.3/183.3 KB 6.8 MB/s eta 0:00:00
  Created wheel for pycountry: filename=pycountry-22.3.5-py2.py3-none-any.whl size=10681832 sha256=127d5a501d6cb42fb18461a5f402ba06d77c3880b88e1136cb97d720a0492392
  Stored in directory: /Users/davidjurajszucs/Library/Caches/pip/wheels/47/15/92/e6dc85fcb0686c82e1edbcfdf80cfe4808c058813fed0baa8f
Successfully built pycountry
Note: you may need to restart the kernel to use updated packages.


In [9]:
import pandas as pd
import numpy as np
import pickle
import country_converter as coco
import pycountry_convert as pc
import geopandas as gdp

In [157]:
def calculate_total_ru_depend(df_ru_import, df_total_import, df_import_depend):
    df_ru_import[['Freq', 'SIEC', 'Partner','Unit','Country']] = df_ru_import['freq,siec,partner,unit,geo\TIME_PERIOD'].str.split(',', expand=True)
    df_ru_import.drop(columns=['freq,siec,partner,unit,geo\TIME_PERIOD', 'Freq', 'Partner', 'Unit', 'SIEC'], inplace=True)

    df_ru_import = df_ru_import.set_index('Country')
    df_ru_import = df_ru_import.replace(': ', np.nan)
    df_ru_import = df_ru_import.astype(np.float64)

    df_total_import[['Freq', 'SIEC', 'Partner','Unit','Country']] = df_total_import['freq,siec,partner,unit,geo\TIME_PERIOD'].str.split(',', expand=True)
    df_total_import.drop(columns=['freq,siec,partner,unit,geo\TIME_PERIOD', 'Freq', 'Partner', 'Unit', 'SIEC'], inplace=True)
    df_total_import = df_total_import.set_index('Country')
    df_total_import = df_total_import.replace(': ', np.nan)
    df_total_import = df_total_import.astype(np.float64)

    df_import_depend[['Freq', 'SIEC','Unit','Country']] = df_import_depend['freq,siec,unit,geo\TIME_PERIOD'].str.split(',', expand=True)
    df_import_depend.drop(columns=['freq,siec,unit,geo\TIME_PERIOD', 'Freq', 'Unit', 'SIEC'], inplace=True)
    df_import_depend = df_import_depend.set_index('Country')
    df_import_depend = df_import_depend.replace(': ', np.nan)
    df_import_depend = df_import_depend.astype(np.float64)
    #print("df_import_dependancy: \n", df_import_depend)

    df_import_relative = df_ru_import.div(df_total_import, fill_value=0)  # This represents how much of the oil imports is from Russia
    df_import_depend_ru = df_import_depend.mul(df_import_relative, fill_value=0)  # This represents total dependency on oil also considering export and local production

    try:
        df_import_depend_ru.drop(['EA19', 'EU28', 'AL', 'BA', 'GE', 'IS', 'MD', 'ME', 'MK', 'NO', 'RS', 'TR', 'UA', 'XK'], axis=0, inplace=True)
        #df_import_depend_ru.drop(['EA19', 'EU27_2020', 'EU28', 'AL', 'BA', 'GE', 'IS', 'MD', 'ME', 'MK', 'NO', 'RS', 'TR', 'UA', 'XK'], axis=0, inplace=True)
    except: # some datasets do not have 'EU28' part
        df_import_depend_ru.drop(['EA19', 'AL', 'BA', 'GE', 'IS', 'MD', 'ME', 'MK', 'NO', 'RS', 'TR', 'UA', 'XK'], axis=0, inplace=True)
        #df_import_depend_ru.drop(['EA19', 'EU27_2020', 'AL', 'BA', 'GE', 'IS', 'MD', 'ME', 'MK', 'NO', 'RS', 'TR', 'UA', 'XK'], axis=0, inplace=True)

    print("df_import_ru_dependancy: \n", df_import_depend_ru)

    df_import_depend_ru.index = df_import_depend_ru.index.str.replace("UK","GB")  # UK and EL not recognised as ISO2
    df_import_depend_ru.index = df_import_depend_ru.index.str.replace("EL","GR")
    #df_import_depend_ru.index = df_import_depend_ru.index.str.slice(-2)
    df_import_depend_ru.index = coco.convert(names=df_import_depend_ru.index.tolist(), to='name_short', not_found=None)
    df_import_depend_ru.index.name = "Country"

    #df_import_depend_ru = df_import_depend_ru.drop(["19"])  # This is EU19 value -> dropping

    df_import_depend_ru = pd.melt(df_import_depend_ru.reset_index(), id_vars=['Country'], value_vars=[(str(2000+i) + " ") for i in range(21)],var_name='Year', value_name='Import').sort_values(['Country', 'Year'])
    df_import_depend_ru["Year"] = pd.to_numeric(df_import_depend_ru["Year"], downcast='integer')
    df_import_depend_ru.reset_index(drop=True, inplace=True)

    return df_import_depend_ru

# 1. Imports of oil and petroleum products by partner country
## a. Dataframe: Imports of oil and petroleum products by partner country (From Russia) 2000-2020
source: https://ec.europa.eu/eurostat/databrowser/view/NRG_TI_OIL__custom_2512547/default/table?lang=en
## b. Dataframe: Imports of oil and petroleum products by partner country (From TOTAL) 2000-2020
source: https://ec.europa.eu/eurostat/databrowser/view/NRG_TI_OIL__custom_2512564/default/table?lang=en
## c. Dataframe: Relative dependency on import from Russia 2000-2020
As values of imports from Russia divided by total imports of oil and petroleum products
## d. Dataframe: Energy imports dependency - oil and petroleum products 2000-2020
source: https://ec.europa.eu/eurostat/databrowser/view/NRG_IND_ID__custom_2512645/default/table?lang=en
Calculated as (imports-exports)/gross_available_energy

In [162]:
df_oil_petrol_ru = pd.read_csv('datasets/nrg_ti_oil__custom_2512547_tabular.tsv', sep="\t" ,header = 0)
df_oil_petrol_total = pd.read_csv('datasets/nrg_ti_oil__custom_2512538_tabular.tsv', sep="\t" ,header = 0)
df_oil_petrol_depend = pd.read_csv('datasets/nrg_ind_id__custom_2512645_tabular.tsv', sep="\t" ,header = 0)
df_oil_petrol_depend_ru = calculate_total_ru_depend(df_oil_petrol_ru, df_oil_petrol_total, df_oil_petrol_depend)
with open('df_oil_petrol_ru.pickle', 'wb') as handle:
    pickle.dump(df_oil_petrol_depend_ru, handle, protocol=pickle.HIGHEST_PROTOCOL)

df_oil_petrol_depend_ru

EU27_2020 not found in regex


df_import_ru_dependancy: 
                2000       2001       2002       2003       2004       2005   \
Country                                                                       
AT          7.003137   7.585507   6.601801   9.825366   7.435077  13.534791   
BE         13.540641  14.139256  21.160970  23.371639  27.235420  27.754928   
BG         82.613866  86.567080  83.032880  58.229609  75.329746  71.270211   
CY         23.478234  19.953139  68.707768  36.763697   3.174573   1.591857   
CZ         53.248362  42.959364  39.873238  43.514540  42.450861  49.295719   
DE         19.856829  20.480505  21.642860  23.957936  25.657120  26.179611   
DK         -2.980812  -4.643169  -5.867252 -10.161986  -6.973978 -10.289990   
EE         51.126783  44.624762  48.395019  38.123644  33.777035  30.559303   
EL         24.029128  31.335703  47.569016  37.412675  32.933258  33.324941   
ES          8.878312   8.873008  12.065723  14.540877  13.106203  14.034871   
EU27_2020  16.924241  19.

,Country,Year,Import
0,Austria,2000,7.003137
1,Austria,2001,7.585507
2,Austria,2002,6.601801
3,Austria,2003,9.825366
4,Austria,2004,7.435077
...,...,...,...
625,United Kingdom,2016,3.221683
626,United Kingdom,2017,4.167897
627,United Kingdom,2018,3.329193
628,United Kingdom,2019,3.406691


# 2. Imports of solid fossil fuels by partner country
## a. Dataframe: Imports of solid fossil fuels by partner country (from Russia) 2000-2020
source: https://ec.europa.eu/eurostat/databrowser/view/NRG_TI_SFF__custom_2512607/default/table?lang=en
## b. Dataframe: Imports of solid fossil fuels by partner country (from TOTAL) 2000-2020
source: https://ec.europa.eu/eurostat/databrowser/view/NRG_TI_SFF__custom_2512600/default/table?lang=en
## c. Dataframe: Relative dependency on import from Russia 2000-2020
As values of imports from Russia divided by total imports of solid fossil fuels
## d. Dataframe: Energy imports dependency - solid fossil fuels 2000-2020
source: https://ec.europa.eu/eurostat/databrowser/view/NRG_IND_ID__custom_2522510/default/table?lang=en
datasets/nrg_ind_id__custom_2522510_tabular.tsv

In [163]:
df_solid_fuel_ru = pd.read_csv('datasets/nrg_ti_sff__custom_2512607_tabular.tsv', sep="\t" ,header = 0)
df_solid_fuel_total = pd.read_csv('datasets/nrg_ti_sff__custom_2512600_tabular.tsv', sep="\t" ,header = 0)
df_solid_fuel_depend = pd.read_csv('datasets/nrg_ind_id__custom_2522510_tabular.tsv', sep="\t" ,header = 0)
df_solid_fuel_depend_ru = calculate_total_ru_depend(df_solid_fuel_ru, df_solid_fuel_total, df_solid_fuel_depend)
df_solid_fuel_depend_ru
with open('df_solid_fuel_ru.pickle', 'wb') as handle:
    pickle.dump(df_solid_fuel_depend_ru, handle, protocol=pickle.HIGHEST_PROTOCOL)

EU27_2020 not found in regex


df_import_ru_dependancy: 
                 2000        2001        2002        2003       2004   \
Country                                                                
AT           0.000000    0.000000    0.000000    0.000000   1.343702   
BE           8.427080   10.190610    9.123953    9.222006  12.524454   
BG           7.210573    0.000000    8.616027   10.991226   9.422147   
CY          61.224600  111.321000  124.528000   96.226000   0.000000   
CZ          -0.077699   -0.036470   -0.332613   -0.432675  -0.550728   
DE           0.903043    2.054760    1.881353    1.973436   4.353752   
DK          19.016506   25.216420   15.215481    9.983268  26.109119   
EE         120.298843   66.356047   90.285475  202.537000  34.173000   
EL           2.189035    3.857166    2.804427    1.537427   3.401487   
ES           4.424416    5.408496    6.510173    5.816537   9.133663   
EU27_2020    2.639144    3.113184    3.433612    3.824954   5.040326   
FI          44.447296   43.548028   3

# 3. Imports of natural gas by partner country
## a. Imports of natural gas by partner country (from Russia)
source: https://ec.europa.eu/eurostat/databrowser/view/NRG_TI_GAS__custom_2512614/default/table?lang=en
## b. Imports of natural gas by partner country (from TOTAL)
source: https://ec.europa.eu/eurostat/databrowser/view/NRG_TI_GAS__custom_2512612/default/table?lang=en
## c. Dataframe: Relative dependency on import from Russia 2000-2020

## d. Dataframe: Energy imports dependency - natural gas 2000-2020
source: https://ec.europa.eu/eurostat/databrowser/view/NRG_IND_ID__custom_2512658/default/table?lang=en

In [164]:
df_nat_gas_ru = pd.read_csv('datasets/nrg_ti_gas__custom_2512614_tabular.tsv', sep="\t" ,header = 0)
df_nat_gas_total = pd.read_csv('datasets/nrg_ti_gas__custom_2512612_tabular.tsv', sep="\t" ,header = 0)
df_nat_gas_depend = pd.read_csv('datasets/nrg_ind_id__custom_2512658_tabular.tsv', sep="\t" ,header = 0)
df_nat_gas_depend_ru = calculate_total_ru_depend(df_nat_gas_ru, df_nat_gas_total, df_nat_gas_depend)

with open('df_nat_gas_ru.pickle', 'wb') as handle:
    pickle.dump(df_nat_gas_depend_ru, handle, protocol=pickle.HIGHEST_PROTOCOL)

df_nat_gas_depend_ru

EU27_2020 not found in regex


df_import_ru_dependancy: 
                 2000        2001        2002        2003        2004   \
Country                                                                 
AT          64.797264   56.735338   55.906068   59.631411   58.038837   
BE           0.000000    0.000000    0.000000    0.000000    2.255428   
BG          93.542000   99.739000  103.995000   94.247000   95.835000   
CY           0.000000    0.000000    0.000000    0.000000    0.000000   
CZ          78.168501   72.513418   74.483574   72.357210   66.999163   
DE          36.207632   32.217287   32.565677   30.993146   34.884842   
DK          -0.000000   -0.000000   -0.000000   -0.000000   -0.000000   
EE         100.000000  100.000000  100.000000  100.000000  100.000000   
EL          74.538351   73.501697   74.160677   76.129515   80.726769   
ES           0.000000    0.000000    0.000000    0.000000    0.000000   
EU27_2020   27.151265   24.487397   25.112543   25.146224   24.848604   
FI         100.000000  1

,Country,Year,Import
0,Austria,2000,64.797264
1,Austria,2001,56.735338
2,Austria,2002,55.906068
3,Austria,2003,59.631411
4,Austria,2004,58.038837
...,...,...,...
625,United Kingdom,2016,0.000000
626,United Kingdom,2017,0.094446
627,United Kingdom,2018,1.601414
628,United Kingdom,2019,3.345057


In [166]:
df_nat_gas_depend_ru.iloc[0]['Country']

'Austria'

## Dataframe: Shapefile of the European countries

In [158]:
#Ref: https://www.eea.europa.eu/data-and-maps/data/eea-coastline-for-analysis-1/gis-data/europe-coastline-shapefile
shapefile = 'datasets/Europe.shp'
gdf = gdp.read_file(shapefile)
gdf = gdf[['NAME','geometry']]
gdf.columns = ['Country','geometry']
with open('gdf.pickle', 'wb') as handle:
    pickle.dump(gdf, handle, protocol=pickle.HIGHEST_PROTOCOL)
gdf

,Country,geometry
0,Albania,"MULTIPOLYGON (((19.50115 40.96230, 19.50563 40..."
1,Andorra,"POLYGON ((1.43992 42.60649, 1.45041 42.60596, ..."
2,Austria,"POLYGON ((16.00000 48.77775, 16.00000 48.78252..."
3,Belgium,"POLYGON ((5.00000 49.79374, 4.99724 49.79696, ..."
4,Bosnia Herzegovina,"POLYGON ((19.22947 43.53458, 19.22925 43.53597..."
5,Croatia,"MULTIPOLYGON (((14.30038 44.50156, 14.28972 44..."
6,Czech Republic,"POLYGON ((14.82523 50.87399, 14.83687 50.86996..."
7,Denmark,"MULTIPOLYGON (((11.99978 54.94118, 11.98534 54..."
8,Estonia,"MULTIPOLYGON (((23.97511 58.09691, 23.96645 58..."
9,Finland,"MULTIPOLYGON (((22.07310 60.22830, 22.06502 60..."


In [160]:
df_yr = df_nat_gas_depend_ru[df_nat_gas_depend_ru['Year'] == 2001]
merged = gdf.merge(df_yr, on='Country')
merged

,Country,geometry,Year,Import
0,Austria,"POLYGON ((16.00000 48.77775, 16.00000 48.78252...",2001,56.735338
1,Belgium,"POLYGON ((5.00000 49.79374, 4.99724 49.79696, ...",2001,0.000000
2,Croatia,"MULTIPOLYGON (((14.30038 44.50156, 14.28972 44...",2001,29.557000
3,Czech Republic,"POLYGON ((14.82523 50.87399, 14.83687 50.86996...",2001,72.513418
4,Denmark,"MULTIPOLYGON (((11.99978 54.94118, 11.98534 54...",2001,-0.000000
5,Estonia,"MULTIPOLYGON (((23.97511 58.09691, 23.96645 58...",2001,100.000000
6,Finland,"MULTIPOLYGON (((22.07310 60.22830, 22.06502 60...",2001,100.000000
7,France,"MULTIPOLYGON (((-2.28137 46.68570, -2.31121 46...",2001,24.031582
8,Germany,"MULTIPOLYGON (((13.11717 54.54924, 13.12529 54...",2001,32.217287
9,Greece,"MULTIPOLYGON (((24.74283 37.59560, 24.73534 37...",2001,73.501697


In [40]:
df_nat_gas_depend_ru

,Country,Year,Import
0,Albania,2000,0.000000
1,Albania,2001,0.000000
2,Albania,2002,0.000000
3,Albania,2003,0.000000
4,Albania,2004,0.000000
...,...,...,...
898,United Kingdom,2016,0.000000
899,United Kingdom,2017,0.094446
900,United Kingdom,2018,1.601414
901,United Kingdom,2019,3.345057


In [10]:
df_sitc = pd.read_csv('datasets/ext_st_eu27_2020sitc_tabular.tsv', sep="\t" , header = 0)
df_sitc[['freq', 'stk_flow', 'indic_et', 'Partner', 'SITC', 'EU']] = df_sitc['freq,stk_flow,indic_et,partner,sitc06,geo\TIME_PERIOD'].str.split(',', expand=True)
del df_sitc['freq,stk_flow,indic_et,partner,sitc06,geo\TIME_PERIOD']; del df_sitc["freq"]; del df_sitc["EU"];
df_sitc = df_sitc.loc[df_sitc['Partner'] == 'RU']
df_sitc = df_sitc.loc[df_sitc['indic_et'] == 'TRD_VAL']
df_sitc = df_sitc.loc[df_sitc['stk_flow'] == 'IMP']
del df_sitc["Partner"]; del df_sitc["indic_et"]; del df_sitc["stk_flow"];
first_column = df_sitc.pop('SITC'); df_sitc.insert(0, 'SITC', first_column)
df_sitc = df_sitc.set_index('SITC').T
#df_sitc = df_sitc.T
#df_sitc = df_sitc.rename(columns={df_sitc.columns[0]: 'date'})
df_sitc = df_sitc.replace(': ', 0)
df_sitc = df_sitc.replace(': n', 0)
df_sitc = df_sitc.astype('float')
#first_column = df_sitc.pop('Partner'); df_sitc.insert(0, 'Partner', first_column)
df_sitc['date'] = df_sitc.index
df_sitc['date'] = pd.to_datetime(df_sitc['date'])
df_sitc = df_sitc.set_index('date')
df_sitc = df_sitc.resample('Y').sum()
df_sitc.index = pd.DatetimeIndex(df_sitc.index).year
#df_sitc = df_sitc.resample('Y', on='date').sum()
#df_sitc.groupby(pd.Grouper(freq='Y'))
#df_sitc.groupby(df_sitc['date'].dt.strftime('%B'))['SITC0'].sum().sort_values()
del df_sitc["TOTAL"]
df_sitc = df_sitc.drop([2022])
df_sitc

/var/folders/87/8h0tk4x16v12b0z4582cy0ch0000gn/T/ipykernel_38657/1761157754.py:1: DtypeWarning: Columns (13,14,15,16,19,24,25,26,27,28,29,30,34,127,129,131,132,133,134,135,136,139,146,147,148,150,151,162,167,173,174,178,179,182,183,184,185,186,188,191,192,193,194,195,198,199,200,203,204,205,206,207,209,212,216,217,218,219,220,223,224,225,231,232) have mixed types. Specify dtype option on import or set low_memory=False.
  df_sitc = pd.read_csv('datasets/ext_st_eu27_2020sitc_tabular.tsv', sep="\t" , header = 0)
/var/folders/87/8h0tk4x16v12b0z4582cy0ch0000gn/T/ipykernel_38657/1761157754.py:8: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  first_column = df_sitc.pop('SITC'); df_sitc.insert(0, 'SITC', first_column)


SITC,SITC0,SITC0_1,SITC1,SITC2,SITC2_4,SITC3,SITC33,SITC4,SITC5,SITC5-8,SITC6,SITC6_8,SITC7,SITC8,SITC9
date,,,,,,,,,,,,,,,
2002,985.0,1021.8,36.9,2607.1,2614.5,37748.7,30209.1,7.6,2161.3,9794.2,6379.3,6694.0,939.3,314.7,9967.3
2003,720.5,759.0,38.4,2834.7,2848.8,41651.9,33108.8,14.2,2413.0,10932.2,7072.9,7386.6,1132.3,313.8,11380.3
2004,524.4,559.1,34.7,4098.2,4122.6,47837.3,40116.0,24.6,2713.2,13510.3,9213.8,9589.6,1207.7,375.7,13253.4
2005,547.9,582.3,34.7,4092.7,4162.2,72244.2,62789.6,69.7,3279.5,14959.1,10121.0,10469.8,1210.1,348.7,14750.9
2006,673.5,717.1,43.7,4614.9,4729.8,91334.8,79698.3,115.0,3609.2,18060.7,12864.3,13286.1,1165.3,422.1,20638.2
2007,776.0,834.9,58.8,4646.6,4802.8,92729.5,81634.0,156.3,3978.3,20209.0,14335.2,14788.4,1442.1,453.0,21551.1
2008,704.4,777.4,73.1,4679.5,4876.6,119872.8,103672.3,197.1,4929.8,20056.9,13311.9,13703.0,1424.1,390.8,26637.5
2009,516.2,580.5,64.3,2163.0,2369.7,85644.1,69362.2,206.5,3081.7,11918.2,7425.7,7698.3,1138.1,273.0,14238.7
2010,469.6,530.9,61.6,3423.6,3620.5,117894.5,98754.5,197.0,4356.3,18023.0,12005.7,12320.9,1345.8,315.0,15728.1


In [11]:
df_sitc.head()

SITC,SITC0,SITC0_1,SITC1,SITC2,SITC2_4,SITC3,SITC33,SITC4,SITC5,SITC5-8,SITC6,SITC6_8,SITC7,SITC8,SITC9
date,,,,,,,,,,,,,,,
2002,985.0,1021.8,36.9,2607.1,2614.5,37748.7,30209.1,7.6,2161.3,9794.2,6379.3,6694.0,939.3,314.7,9967.3
2003,720.5,759.0,38.4,2834.7,2848.8,41651.9,33108.8,14.2,2413.0,10932.2,7072.9,7386.6,1132.3,313.8,11380.3
2004,524.4,559.1,34.7,4098.2,4122.6,47837.3,40116.0,24.6,2713.2,13510.3,9213.8,9589.6,1207.7,375.7,13253.4
2005,547.9,582.3,34.7,4092.7,4162.2,72244.2,62789.6,69.7,3279.5,14959.1,10121.0,10469.8,1210.1,348.7,14750.9
2006,673.5,717.1,43.7,4614.9,4729.8,91334.8,79698.3,115.0,3609.2,18060.7,12864.3,13286.1,1165.3,422.1,20638.2


In [13]:
with open('df_sitc.pickle', 'wb') as handle:
    pickle.dump(df_sitc, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [21]:
years_int = df_sitc.index.tolist()
years = list(map(str, years_int))
years

['2002',
 '2003',
 '2004',
 '2005',
 '2006',
 '2007',
 '2008',
 '2009',
 '2010',
 '2011',
 '2012',
 '2013',
 '2014',
 '2015',
 '2016',
 '2017',
 '2018',
 '2019',
 '2020',
 '2021']

In [57]:
with open('df_sitc.pickle', 'rb') as handle:
    df_sitc = pickle.load(handle)

df_sitc

df_sitc.columns.values.tolist()
print(df_sitc.columns.values)
df_sitc = df_sitc.reset_index()
df_sitc['date'] = df_sitc['date'].astype(str) # to show it in the x-axis
df_sitc

['SITC0' 'SITC0_1' 'SITC1' 'SITC2' 'SITC2_4' 'SITC3' 'SITC33' 'SITC4'
 'SITC5' 'SITC5-8' 'SITC6' 'SITC6_8' 'SITC7' 'SITC8' 'SITC9']


SITC,date,SITC0,SITC0_1,SITC1,SITC2,SITC2_4,SITC3,SITC33,SITC4,SITC5,SITC5-8,SITC6,SITC6_8,SITC7,SITC8,SITC9
0,2002,985.0,1021.8,36.9,2607.1,2614.5,37748.7,30209.1,7.6,2161.3,9794.2,6379.3,6694.0,939.3,314.7,9967.3
1,2003,720.5,759.0,38.4,2834.7,2848.8,41651.9,33108.8,14.2,2413.0,10932.2,7072.9,7386.6,1132.3,313.8,11380.3
2,2004,524.4,559.1,34.7,4098.2,4122.6,47837.3,40116.0,24.6,2713.2,13510.3,9213.8,9589.6,1207.7,375.7,13253.4
3,2005,547.9,582.3,34.7,4092.7,4162.2,72244.2,62789.6,69.7,3279.5,14959.1,10121.0,10469.8,1210.1,348.7,14750.9
4,2006,673.5,717.1,43.7,4614.9,4729.8,91334.8,79698.3,115.0,3609.2,18060.7,12864.3,13286.1,1165.3,422.1,20638.2
5,2007,776.0,834.9,58.8,4646.6,4802.8,92729.5,81634.0,156.3,3978.3,20209.0,14335.2,14788.4,1442.1,453.0,21551.1
6,2008,704.4,777.4,73.1,4679.5,4876.6,119872.8,103672.3,197.1,4929.8,20056.9,13311.9,13703.0,1424.1,390.8,26637.5
7,2009,516.2,580.5,64.3,2163.0,2369.7,85644.1,69362.2,206.5,3081.7,11918.2,7425.7,7698.3,1138.1,273.0,14238.7
8,2010,469.6,530.9,61.6,3423.6,3620.5,117894.5,98754.5,197.0,4356.3,18023.0,12005.7,12320.9,1345.8,315.0,15728.1
9,2011,863.7,911.8,48.1,3940.1,4162.9,147986.3,125821.8,223.0,5319.8,20843.2,13497.4,13851.0,1672.1,353.7,19154.2


# Extended dataset for all exporting countries
## Partner country codes in the Eurostat dataframes

[NSP] Not specified
[AFR_OTH] Other African countries (aggregate changing according to the context)
[AME_OTH] Other American countries (aggregate changing according to the context)
[ASI_OTH] Other Asian countries (aggregate changing according to the context)
[EX_SU_OTH] Other countries of former Soviet Union (before 1991)
[EUR_OTH] Other European countries (aggregate changing according to the context)
[ASI_NME_OTH] Other Near and Middle East Asian countries

In [38]:
def country_code_to_continent(cc):
    if cc == 'UK': cc = 'GB'
    if cc == 'EL': cc = 'GR'
    if cc == 'TL': cc = 'TP'
    try:
        continent = pc.country_alpha2_to_continent_code(cc)
    except KeyError:
            continent = 'Unknown'
    if continent == 'Unknown':
        if cc == 'AFR_OTH': continent = 'AF'
        if cc == 'AME_OTH': continent = 'SA'
        if cc == 'ASI_OTH': continent = 'AS'
        if cc == 'EX_SU_OTH': continent = 'AS'
        if cc == 'EUR_OTH': continent = 'EU'
        if cc == 'ASI_NME_OTH': continent = 'AS'
    return continent

In [98]:
def continent_code_to_name(cc):
    if cc == 'AF': continent = 'Africa'
    elif cc == 'AS': continent = 'Asia'
    elif cc == 'EU': continent = 'Europe'
    elif cc == 'NA': continent = 'North America'
    elif cc == 'OC': continent = 'Oceania'
    elif cc == 'SA': continent = 'South America'
    else: continent = 'Unknown'
    return continent

In [110]:
def dataset_for_treemap(df_extended):
    df_extended[['Freq', 'SIEC', 'Partner','Unit','Country']] = df_extended['freq,siec,partner,unit,geo\TIME_PERIOD'].str.split(',', expand=True)
    df_extended.drop(columns=['freq,siec,partner,unit,geo\TIME_PERIOD', 'Freq', 'Unit', 'SIEC'], inplace=True)

    df_extended = df_extended.set_index('Country')
    df_extended = df_extended.replace(': ', np.nan)
    #df_extended.sort_values('Country', inplace=True)
    try:
        df_extended.drop(['EA19', 'EU28', 'AL', 'BA', 'GE', 'IS', 'MD', 'ME', 'MK', 'NO', 'RS', 'TR', 'UA', 'XK'], axis=0, inplace=True)
    except: # some datasets do not have 'EU28' part
        df_extended.drop(['EA19', 'AL', 'BA', 'GE', 'IS', 'MD', 'ME', 'MK', 'NO', 'RS', 'TR', 'UA', 'XK'], axis=0, inplace=True)

    df_extended['Continent'] = df_extended['Partner'].apply(lambda x: continent_code_to_name(country_code_to_continent(x)))

    cols = df_extended.columns.tolist()
    cols = cols[-1:] + cols[-2:-1] + cols[:-2]  # Changing the order of the columns
    df_extended = df_extended[cols]

    df_extended.reset_index(inplace=True)
    df_extended.drop(df_extended[df_extended['Continent'] == 'Unknown'].index, inplace = True)

    df_extended.sort_values(['Country', 'Continent', 'Partner'], inplace=True)
    df_extended = df_extended.replace(['EL'],'GR')
    df_extended = df_extended.replace(['UK'],'GB')
    df_extended['Country'] = coco.convert(names=df_extended['Country'].tolist(), to='name_short', not_found=None)
    df_extended['Partner'] = coco.convert(names=df_extended['Partner'].tolist(), to='name_short', not_found=None)
    #df_extended = df_extended.set_index(['Country', 'Continent', 'Partner'])
    #df_extended.reset_index(inplace=True)
    df_extended = pd.melt(df_extended, id_vars=['Country', 'Continent', 'Partner'], value_vars=[(str(2000+i) + " ") for i in range(21)], var_name='Year', value_name='Import')

    df_extended = df_extended[['Country', 'Year', 'Continent', 'Partner', 'Import']].sort_values(['Country', 'Year', 'Continent', 'Partner'])  # Changing the order of the columns
    df_extended["Year"] = pd.to_numeric(df_extended["Year"], downcast='integer')
    df_extended["Import"] = df_extended["Import"].astype(np.float64)
    df_extended.reset_index(drop=True, inplace=True)

    return df_extended

OIL EXPORTER EXTENDED

In [112]:
df_extended_oil = pd.read_csv('datasets/nrg_ti_oil__custom_2743167_tabular.tsv', sep="\t" ,header = 0)
df_oil_petrol_exporters = dataset_for_treemap(df_extended_oil)
print(df_oil_petrol_exporters)
with open('df_oil_petrol_exporters.pickle', 'wb') as handle:
    pickle.dump(df_oil_petrol_exporters, handle, protocol=pickle.HIGHEST_PROTOCOL)

EU27_2020 not found in regex
EU27_2020 not found in regex
EU27_2020 not found in regex
EU27_2020 not found in regex
EU27_2020 not found in regex
EU27_2020 not found in regex
EU27_2020 not found in regex
EU27_2020 not found in regex
EU27_2020 not found in regex
EU27_2020 not found in regex
EU27_2020 not found in regex
EU27_2020 not found in regex
EU27_2020 not found in regex
EU27_2020 not found in regex
EU27_2020 not found in regex
EU27_2020 not found in regex
EU27_2020 not found in regex
EU27_2020 not found in regex
EU27_2020 not found in regex
EU27_2020 not found in regex
EU27_2020 not found in regex
EU27_2020 not found in regex
EU27_2020 not found in regex
EU27_2020 not found in regex
EU27_2020 not found in regex
EU27_2020 not found in regex
EU27_2020 not found in regex
EU27_2020 not found in regex
EU27_2020 not found in regex
EU27_2020 not found in regex
EU27_2020 not found in regex
EU27_2020 not found in regex
EU27_2020 not found in regex
EU27_2020 not found in regex
EU27_2020 not 

               Country  Year      Continent     Partner  Import
0              Austria  2000         Africa     AFR_OTH     0.0
1              Austria  2000         Africa     Algeria   739.0
2              Austria  2000         Africa      Angola     0.0
3              Austria  2000         Africa       Benin     0.0
4              Austria  2000         Africa  Cabo Verde     0.0
...                ...   ...            ...         ...     ...
102685  United Kingdom  2020  South America    Colombia     NaN
102686  United Kingdom  2020  South America     Ecuador     NaN
102687  United Kingdom  2020  South America        Peru     NaN
102688  United Kingdom  2020  South America     Uruguay     NaN
102689  United Kingdom  2020  South America   Venezuela     NaN

[102690 rows x 5 columns]


SOLID FUELS EXPORTER EXTENDED

In [115]:
df_extended_solid_fuel = pd.read_csv('datasets/nrg_ti_sff__custom_2757581_tabular.tsv', sep="\t" ,header = 0)
df_solid_fuel_exporters = dataset_for_treemap(df_extended_solid_fuel)
print(df_solid_fuel_exporters)
with open('df_solid_fuel_exporters.pickle', 'wb') as handle:
    pickle.dump(df_solid_fuel_exporters, handle, protocol=pickle.HIGHEST_PROTOCOL)

EU27_2020 not found in regex
EU27_2020 not found in regex
EU27_2020 not found in regex
EU27_2020 not found in regex
EU27_2020 not found in regex
EU27_2020 not found in regex
EU27_2020 not found in regex
EU27_2020 not found in regex
EU27_2020 not found in regex
EU27_2020 not found in regex
EU27_2020 not found in regex
EU27_2020 not found in regex
EU27_2020 not found in regex
EU27_2020 not found in regex
EU27_2020 not found in regex
EU27_2020 not found in regex
EU27_2020 not found in regex
EU27_2020 not found in regex
EU27_2020 not found in regex
EU27_2020 not found in regex
EU27_2020 not found in regex
EU27_2020 not found in regex
EU27_2020 not found in regex
EU27_2020 not found in regex
EU27_2020 not found in regex
EU27_2020 not found in regex
EU27_2020 not found in regex
EU27_2020 not found in regex
EU27_2020 not found in regex
EU27_2020 not found in regex
EU27_2020 not found in regex
EU27_2020 not found in regex
EU27_2020 not found in regex
EU27_2020 not found in regex
EU27_2020 not 

               Country  Year      Continent     Partner  Import
0              Austria  2000         Africa     AFR_OTH     0.0
1              Austria  2000         Africa     Algeria     0.0
2              Austria  2000         Africa      Angola     0.0
3              Austria  2000         Africa       Benin     0.0
4              Austria  2000         Africa  Cabo Verde     0.0
...                ...   ...            ...         ...     ...
102685  United Kingdom  2020  South America    Colombia     NaN
102686  United Kingdom  2020  South America     Ecuador     NaN
102687  United Kingdom  2020  South America        Peru     NaN
102688  United Kingdom  2020  South America     Uruguay     NaN
102689  United Kingdom  2020  South America   Venezuela     NaN

[102690 rows x 5 columns]


NATURAL GAS EXPORTER EXTENDED

In [116]:
df_extended_natural_gas = pd.read_csv('datasets/nrg_ti_gas__custom_2757589_tabular.tsv', sep="\t" ,header = 0)
df_natural_gas_exporters = dataset_for_treemap(df_extended_natural_gas)
print(df_natural_gas_exporters)
with open('df_natural_gas_exporters.pickle', 'wb') as handle:
    pickle.dump(df_natural_gas_exporters, handle, protocol=pickle.HIGHEST_PROTOCOL)

EU27_2020 not found in regex
EU27_2020 not found in regex
EU27_2020 not found in regex
EU27_2020 not found in regex
EU27_2020 not found in regex
EU27_2020 not found in regex
EU27_2020 not found in regex
EU27_2020 not found in regex
EU27_2020 not found in regex
EU27_2020 not found in regex
EU27_2020 not found in regex
EU27_2020 not found in regex
EU27_2020 not found in regex
EU27_2020 not found in regex
EU27_2020 not found in regex
EU27_2020 not found in regex
EU27_2020 not found in regex
EU27_2020 not found in regex
EU27_2020 not found in regex
EU27_2020 not found in regex
EU27_2020 not found in regex
EU27_2020 not found in regex
EU27_2020 not found in regex
EU27_2020 not found in regex
EU27_2020 not found in regex
EU27_2020 not found in regex
EU27_2020 not found in regex
EU27_2020 not found in regex
EU27_2020 not found in regex
EU27_2020 not found in regex
EU27_2020 not found in regex
EU27_2020 not found in regex
EU27_2020 not found in regex
EU27_2020 not found in regex
EU27_2020 not 

               Country  Year      Continent     Partner  Import
0              Austria  2000         Africa     AFR_OTH     0.0
1              Austria  2000         Africa     Algeria     0.0
2              Austria  2000         Africa      Angola     0.0
3              Austria  2000         Africa       Benin     0.0
4              Austria  2000         Africa  Cabo Verde     0.0
...                ...   ...            ...         ...     ...
102685  United Kingdom  2020  South America    Colombia     NaN
102686  United Kingdom  2020  South America     Ecuador     NaN
102687  United Kingdom  2020  South America        Peru     NaN
102688  United Kingdom  2020  South America     Uruguay     NaN
102689  United Kingdom  2020  South America   Venezuela     NaN

[102690 rows x 5 columns]


In [117]:
df_natural_gas_exporters

,Country,Year,Continent,Partner,Import
0,Austria,2000,Africa,AFR_OTH,0.0
1,Austria,2000,Africa,Algeria,0.0
2,Austria,2000,Africa,Angola,0.0
3,Austria,2000,Africa,Benin,0.0
4,Austria,2000,Africa,Cabo Verde,0.0
...,...,...,...,...,...
102685,United Kingdom,2020,South America,Colombia,NaN
102686,United Kingdom,2020,South America,Ecuador,NaN
102687,United Kingdom,2020,South America,Peru,NaN
102688,United Kingdom,2020,South America,Uruguay,NaN
